In [1]:
import sklearn.datasets as datasets
import numpy as np
import scipy

In [2]:
def load_iris():
    return datasets.load_iris()['data'].T, datasets.load_iris()['target']

def vcol(v):
    return v.reshape(-1, 1)

def vrow(v):
    return v.reshape(1, -1)

In [3]:
def cov(D, mu = False):
    mu_ = D.mean(axis=1)
    DC = D - vcol(mu_)
    C = np.dot(DC, DC.T) / D.shape[1]
    if mu:
        return C, mu_
    return C

def split(D, L, seed=0):
    nTrain = int(D.shape[1] * 2.0 / 3.0)
    np.random.seed(seed)
    idx = np.random.permutation(D.shape[1])
    idxTrain = idx[0:nTrain]
    idxTest = idx[nTrain:]
    DTR = D[:, idxTrain]
    DVAL = D[:, idxTest]
    LTR = L[idxTrain]
    LVAL = L[idxTest]
    return (DTR, LTR), (DVAL, LVAL)

In [4]:
import numpy as np


def logpdf_GAU_ND_sing(x, mu, C):
    """
    x: data matrix of shape (M, 1)
    mu: numpy array of shape (M, 1)
    C: numpy array of shape (M, M) representing the covariance matrix
    """
    M = x.shape[0]
    C_inv = np.linalg.inv(C)
    sign, slogdet = np.linalg.slogdet(C)
    C_inv, sign, slogdet
    factor1 = -(M / 2) * np.log(2 * np.pi)
    factor2 = -0.5 * slogdet
    factor3 = -0.5 * np.dot(np.dot((x - mu).T, C_inv), (x - mu))

    return factor1 + factor2 + factor3


def logpdf_GAU_ND(x, mu, C):
    """
    x: data matrix of shape (M, n)
    mu: numpy array of shape (M, 1)
    C: numpy array of shape (M, M) representing the covariance matrix
    """
    M = x.shape[0]
    C_inv = np.linalg.inv(C)  # shape (M, M)
    _, slogdet = np.linalg.slogdet(C)
    factor1 = -(M / 2) * np.log(2 * np.pi)
    factor2 = -0.5 * slogdet

    diff = x.T - mu  # x.T shape (n, M), mu.T shape (1, M), diff shape (n, M)

    product = np.dot(C_inv, diff.T)
    # C_inv shape (M, M), diff.T shape (M, n), product shape (M, n)
    factor3 = -0.5 * np.sum(diff.T * product, axis=0)
    # diff.T shape (M, n), product shape (M, n), result shape (n,)
    return factor1 + factor2 + factor3


def loglikelihood(x, mu, C):
    return np.sum(logpdf_GAU_ND(x, mu, C))


In [5]:
def within_cov(D, labels):
    classes = set(labels)
    C = np.zeros((D.shape[0], D.shape[0]))
    for class_ in classes:
        Dc = D[:, labels==class_]
        mu_c = Dc.mean(axis=1)
        Dc -= vcol(mu_c)
        C += np.dot(Dc, Dc.T)
    return C/D.shape[1]

In [6]:
data, labels = load_iris()

In [7]:
(DTR, LTR), (DVAL, LVAL) = split(data, labels)

In [24]:
def gaussian_classifier(DTR, LTR, DVAL, LVAL, prior, type = 0):
    """
    type 0: MVG
    type 1: Naive Bayes
    type 2: Tied
    """
    mus = [DTR[:, LTR == i].mean(axis=1) for i in set(LTR)]
    Cs = [cov(DTR[:, LTR == i]) for i in set(LTR)]
    if type == 1:
        Cs = [C * np.identity(C.shape[0]) for C in Cs]
    if type == 2:
        C_w = within_cov(DTR, LTR)
        Cs = [C_w for _ in range(len(mus))]
    densities = np.array([np.exp(logpdf_GAU_ND(DVAL, mus[i], Cs[i])) for i in range(len(mus))])
    SJoint = densities * prior
    SMargin = vrow(SJoint.sum(axis=0))
    SPost = SJoint / SMargin
    print(np.argmax(SPost, axis=0))
    return 1-np.sum(np.argmax(SPost, axis=0) == LVAL)/LVAL.shape[0]

In [29]:
def binary_gaussian_classifier(DTR, LTR, DVAL, LVAL, prior, type = 0):
    """
    type 0: MVG
    type 1: Naive Bayes
    type 2: Tied
    """
    mus = [DTR[:, LTR == i].mean(axis=1) for i in set(LTR)]
    Cs = [cov(DTR[:, LTR == i]) for i in set(LTR)]
    if type == 1:
        Cs = [C * np.identity(C.shape[0]) for C in Cs]
    if type == 2:
        C_w = within_cov(DTR, LTR)
        Cs = [C_w for _ in range(len(mus))]
    densities = np.array([np.exp(logpdf_GAU_ND(DVAL, mus[i], Cs[i])) for i in range(len(mus))])
    llr = densities[1] - densities[0]
    predictions = llr >= 0
    return 1-np.sum(predictions == LVAL)/LVAL.shape[0]

In [21]:
gaussian_classifier(DTR, LTR, DVAL, LVAL, 1/3, type=0), gaussian_classifier(DTR, LTR, DVAL, LVAL, 1/3, type=1), gaussian_classifier(DTR, LTR, DVAL, LVAL, 1/3, type=2)

(4, 100) (100,) (4, 50) (50,)
(4, 100) (100,) (4, 50) (50,)
(4, 100) (100,) (4, 50) (50,)


(0.040000000000000036, 0.040000000000000036, 0.020000000000000018)

In [22]:
b_data = data[:, labels != 0]
b_labels = labels[labels != 0]
(bDTR, bLTR), (bDVAL, bLVAL) = split(b_data, b_labels)

In [30]:
gaussian_classifier(bDTR, bLTR, bDVAL, bLVAL-1, 1/2, type=0), binary_gaussian_classifier(bDTR, bLTR, bDVAL, bLVAL-1, 1/2, type=0)

[0 1 0 1 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 1 1 0 1 1 1 0 0 0 0 1 1 1 0 0]


(0.08823529411764708, 0.08823529411764708)